# Main script to clean wind data at the zip code, monthly level

Modules: N/A <br>
Author: Cornelia Ilin <br>
Email: cilin@wisc.edu <br>
Date created: May 14, 2021 <br>

**Citations (data sources)**

``Wind data:`` 

download the "MERRA2_100.tavgM_2d_slv_Nx" product; this provides monthly averages of U and V components

1. https://search.earthdata.nasa.gov/search/granules?p=C1276812859-GES_DISC&pg[0][qt]=1991-01-01T00%3A00%3A00.000Z%2C2017-12-31T23%3A59%3A59.999Z&pg[0][gsk]=-start_date&q=MERRA-2%20tavgM&tl=1624239533!3!!&m=-0.0703125!0.0703125!2!1!0!0%2C2

__Jordan steps for wind data__ 
  * Search for report "M2TMNXSLV"
  * Narrow down scope with geoshape file for CA (NOT ZCTA)  
      * Acquired here https://data.ca.gov/dataset/ca-geographic-boundaries 
        * **No VPN access seems to be permitted here**
  * Query for 1991-01-01 00:00:00 - Today  
      * Click recurring

and data dictionary here:

2. https://gmao.gsfc.nasa.gov/pubs/docs/Bosilovich785.pdf
3. https://disc.gsfc.nasa.gov/datasets/M2T1NXSLV_5.12.4/summary


``Shapefiles for California ZIP codes (2010 census):``

4. https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2010&layergroup=ZIP+Code+Tabulation+Areas

__Jordan tweaks__
  * There is an updated 2020 and 2022 ZCTA file available but I think it makes sense to keep the 2010 as that is what project started with and shouldn't have changed much

``Installation errors with Geopandas:``

5. https://stackoverflow.com/questions/54734667/error-installing-geopandas-a-gdal-api-version-must-be-specified-in-anaconda

``How to compute wind speed and direction:``

6. https://stackoverflow.com/questions/21484558/how-to-calculate-wind-direction-from-u-and-v-wind-components-in-r
7. https://github.com/blaylockbk/Ute_WRF/blob/master/functions/wind_calcs.py

``Wind speed and direction intuition:``

8. http://colaweb.gmu.edu/dev/clim301/lectures/wind/wind-uv
9. https://www.earthdatascience.org/courses/use-data-open-source-python/intro-vector-data-python/spatial-data-vector-shapefiles/intro-to-coordinate-reference-systems-python/

``To create maps of this wind data:``

and also used to provide intuition for winddir and windspeed

10. https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20calculate%20and%20plot%20wind%20speed%20using%20MERRA-2%20wind%20component%20data%20using%20Python


``Error - MultiPolygon or multipoly is not iterable``  

This seems to come from an error in version of shapely. Force install with a version below 2.0, I use `shapely==1.8.5` in my env.


**Citations (persons)**
1. N/A

**Preferred environment**
1. Code written in Jupyter Notebooks

### Step 1: Import packages

In [73]:
import pandas as pd
import numpy as np
import netCDF4 as ncdf
import os
from datetime import date, timedelta
from math import pi
import fiona

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

# geography
import geopandas as gpd
import osmnx as ox
import shapely
from shapely.geometry import Point

#Moved from sklearn.neighbors to sklearn.metrics following their package change
import sklearn.metrics
dist = sklearn.metrics.DistanceMetric.get_metric(
    'haversine'
)

# ignore warnings
import warnings
warnings.filterwarnings(
    'ignore'
)

### Step 2: Define working directories

In [74]:
# in_dir_zip_shapes = 'C:/Users/cilin/Research/CA_hospitals/Input/raw_data/census_geo/shapefiles_zcta/'
# in_dir = 'C:/Users/cilin/Research/CA_hospitals/Input/raw_data/winds/'
# in_health = 'C:/Users/cilin/Research/CA_hospitals/Input/final_data/health/'
# out_dir = 'C:/Users/cilin/Research/CA_hospitals/Input/final_data/winds/'

#Local directories on my machine (not gdrive)
in_dir_zip_shapes = 'wind/tl_2010_06_zcta510/'
in_dir = 'wind/ca only/'
in_health = 'health/'
out_dir = 'wind/clean/'

In [82]:
%tree

UsageError: Line magic function `%tree` not found.


### Step 3: Define functions

``read_clean wind``

In [83]:
def read_clean_wind(year):
    """
    Read in wind data by year
    """
    # create empty df
    df = pd.DataFrame()

    for file in os.listdir(in_dir):
        if file.startswith('MERRA2') and file[-10:-6] == str(year):

            ## read .nc file ##
            ###################
            data = ncdf.Dataset(
                in_dir + file, mode='r'
            )
            # print metadata
            #print(data)

            # grab vars of interest ##
            ##########################
            # longitude and latitude
            lons = data.variables['lon']
            lats = data.variables['lat']
            # 2-meter eastward wind m/s
            U2M = data.variables['U2M']
            # 2-meter northward wind m/s
            V2M = data.variables['V2M']

            # Replace vals #
            ################
            #\_FillValues with NaNs:
            U2M_nans = U2M[:]
            V2M_nans = V2M[:]
            _FillValueU2M = U2M._FillValue
            _FillValueV2M = V2M._FillValue
            U2M_nans[U2M_nans == _FillValueU2M] = np.nan
            V2M_nans[V2M_nans == _FillValueV2M] = np.nan

            # Add new vars #
            ################
            # calculate wind speed
            wspd = np.sqrt(U2M_nans**2+V2M_nans**2)

            # calculate wind direction in radians
            wdir = np.arctan2(V2M_nans, U2M_nans)
            #WDIR= (270-atan2(V,U)*180/pi)%360
            
            # transform wind direction from radians to degrees
            #dir_to_degrees = np.mod(180+np.rad2deg(np.arctan2(V2M_nans, U2M_nans)), 360) # this computes "wind is blowing from"' meteorological convetion'
            wdir_to_degrees = np.mod(np.rad2deg(wdir), 360) # this computes "wind is blowing towards" 'oceonographic convention', see here: https://www.esri.com/arcgis-blog/products/product/analytics/displaying-speed-and-direction-symbology-from-u-and-v-vectors/
            
            
            ## transform to df ##
            #####################
            # create an empty df for wind speed and direction with size len(lats) x len(lons) 
            df_wdir = pd.DataFrame(index=lats[:], columns=lons[:])   
            df_wspd = pd.DataFrame(index=lats[:], columns=lons[:])
            
            # create an empty df for u and v components with size len(lats) x len(lons) 
            df_u = pd.DataFrame(index=lats[:], columns=lons[:])
            df_v = pd.DataFrame(index=lats[:], columns=lons[:])

            # populate each row in the empty df above with the wdir_meteo and wspd data and u and v components
            for idx, idx_val in enumerate(df_wdir.index):
                df_wdir.loc[idx_val, :] = wdir_to_degrees[0][idx]
                df_wspd.loc[idx_val, :] = wspd[0][idx]
                df_u.loc[idx_val, :] = U2M_nans[0][idx]
                df_v.loc[idx_val, :] = V2M_nans[0][idx]

            # add index (latitude) as column
            df_wdir.reset_index(
                drop=False,
                inplace=True
            )
            
            df_wdir.rename(
                columns={'index':'lat'},
                inplace=True
            )
            
            
            df_wspd.reset_index(
                drop=False,
                inplace=True
            )
            
            df_wspd.rename(
                columns={'index':'lat'},
                inplace=True
            )
            
            df_u.reset_index(
                drop=False,
                inplace=True
            )
            
            df_u.rename(
                columns={'index':'lat'},
                inplace=True
            )
            
            df_v.reset_index(
                drop=False,
                inplace=True
            )
            
            df_v.rename(
                columns={'index':'lat'},
                inplace=True
            )

            # transform from wide to long
            df_wdir = pd.melt(
                df_wdir, id_vars='lat',
                var_name='lon',
                value_vars=lons[:],
                value_name='wdir'
            )
            
            df_wspd = pd.melt(
                df_wspd,
                id_vars='lat',
                var_name='lon',
                value_vars=lons[:],
                value_name='wspd'
            )
            
            df_u = pd.melt(
                df_u, id_vars='lat',
                var_name='lon',
                value_vars=lons[:],
                value_name='u'
            )
            
            df_v = pd.melt(
                df_v, id_vars='lat',
                var_name='lon',
                value_vars=lons[:],
                value_name='v'
            )

            # concatenate df_wdir and df_wspd
            df_temp1 = df_wdir.merge(
                df_wspd,
                on=['lat', 'lon'],
                how='left'
            )
            
            # concatenate df_u and df_v
            df_temp2 = df_u.merge(
                df_v,
                on=['lat', 'lon'],
                how='left'
            )
            
            # concatenate df_temp1 and df_temp2
            df_temp = df_temp2.merge(
                df_temp1,
                on=['lat', 'lon'],
                how='left'
            )
            
            # add time stamp 
            df_temp['year_month'] = file.split('.')[2]

            df = pd.concat(
                [df_temp, df],
                axis=0
            )
   
    # keep values in min, max range of California geometry
    df = df[
        df.lon.ge(-125) & df.lon.le(-115) & df.lat.ge(32) & df.lat.le(42)
    ]
    
    # transform vars
    df['lat'] = df.lat.astype(float)
    df['lon'] = df.lon.astype(float)
    
    return df

``read census geom``

In [77]:
def read_census_geom():
    """ Read Census (lat, lon) coordinates for California zip-codes
    parameters:
    -----------
    None
    
    return:
    -------
    Df with osmnx_geom
    """
    ### Step 1 ### 
    ##############
    # Read the shapefiles for California's ZIP codes
    for file in os.listdir(in_dir_zip_shapes):
        if file.endswith('.shp'):
            gdf = gpd.read_file(in_dir_zip_shapes + file)

    # keep only cols of interest 
    # ('ZCTA5CE10' = 2010 Census ZIP codes,	'GEOID10' = 2010 Census Tract codes)
    gdf = gdf[
        ['ZCTA5CE10',
         'GEOID10',
         'geometry']
    ]
    
    
    ### Step 2 ###
    ###############
    # For each zip cpde extract polygon with (lat, lon) info

    zip_poly = pd.DataFrame()

    for idx, multipoly in enumerate(gdf.geometry):
        if isinstance(multipoly, shapely.geometry.polygon.Polygon):
            temp_df = pd.DataFrame(
                {
                    'lat': multipoly.exterior.coords.xy[1], 
                    'lon': multipoly.exterior.coords.xy[0],
                    'ZCTA10': gdf.loc[idx, 'ZCTA5CE10'],
                    'GEOID10': gdf.loc[idx, 'GEOID10']
                }
            )
            zip_poly = pd.concat(
                [zip_poly, temp_df],
                axis=0
            )

        if isinstance(multipoly, shapely.geometry.multipolygon.MultiPolygon):
            for poly in multipoly:
                temp_df = pd.DataFrame(
                    {
                        'lat': poly.exterior.coords.xy[1], 
                        'lon': poly.exterior.coords.xy[0],
                        'ZCTA10': gdf.loc[idx, 'ZCTA5CE10'],
                        'GEOID10': gdf.loc[idx, 'GEOID10']
                    }
                )
                zip_poly = pd.concat(
                    [zip_poly, temp_df],
                    axis=0
                )   
    

    # round (lat, lon) to 2 decimal points and add 0.005 to match the UW (lat, lon) values
    zip_poly['lat'] = zip_poly.lat.round(3)
    zip_poly['lon'] = zip_poly.lon.round(3)
    
    zip_poly.sort_values(
        by=['ZCTA10', 'lat', 'lon'],
        inplace=True
    )
    
    zip_poly.drop_duplicates(
        subset=['ZCTA10', 'lat', 'lon'],
        inplace=True
    )

    zip_poly.reset_index(
        drop=True,
        inplace=True
    )
    
    return zip_poly

``find zip (zcta) code for wind data``

In [78]:
def add_zcta_to_wind(df1, df2):
    '''
    params:
    -------
    df1: wind data
    df2: census geometry data
    
    return:
    -------
    '''
    
    # create labels
    df1['wind_lat_lon'] = [str(xy) for xy in zip(df1.lat, df1.lon)]
    df2['census_lat_lon'] = [str(xy) for xy in zip(df2.lat, df2.lon)]

    ## for each point in wind data find the nearest point in the census data ##
    ###############
    # keep only unique points in wind data
    df1_unique = df1.drop_duplicates(
        ['wind_lat_lon']
    )
    
    df2_unique = df2.drop_duplicates(
        ['census_lat_lon']
    )
    
    df1_unique.reset_index(
        drop=True,
        inplace=True
    )
    
    df2_unique.reset_index(
        drop=True,
        inplace=True
    )

    # transform to radians
    df1_unique['lat_r'] = np.radians(df1_unique.lat)
    df1_unique['lon_r'] = np.radians(df1_unique.lon)
    df2_unique['lat_r'] = np.radians(df2_unique.lat)
    df2_unique['lon_r'] = np.radians(df2_unique.lon)


    # compute pairwise distance (in miles)
    dist_matrix = (dist.pairwise(
        df2_unique[['lat_r', 'lon_r']],
        df1_unique[['lat_r', 'lon_r']]
    ))*3959

    # create a df from dist_matrix
    dist_matrix = pd.DataFrame(
        dist_matrix,
        index=df2_unique['census_lat_lon'],
        columns=df1_unique['wind_lat_lon']
    )
    
    # for each row (census_lat_lon point) extract the closest column (wind_lat_lon point) 
    closest_point = pd.DataFrame(
        dist_matrix.idxmin(axis=1),
        columns=['closest_wind_lat_lon']
    )
    
    closest_point.reset_index(
        drop=False,
        inplace=True
    )

    # merge with census data
    df2_unique = df2_unique.merge(
        closest_point,
        on='census_lat_lon',
        how='left'
    )
    
    # merge with census data 
    df2_unique = df2_unique.merge(
        df2[['census_lat_lon']],
        on=['census_lat_lon'],
        how='left'
    )
    
    # replicate df2_unique based on number of year_month entries in df1
    df2_unique = pd.concat(
        [df2_unique]*(df1.year_month.nunique()),
        axis=0
    )
    
    df2_unique.reset_index(
        drop=True,
        inplace=True
    )
    
    # add year_month column to df2_unique
    df2_unique['year_month'] = 0
    indeces = [n for n in range(1, df2_unique.shape[0]) if n%956926==0]

    year_month = np.sort(df1.year_month.unique())
    for idx, index in enumerate(indeces):
        if idx==0:
            df2_unique.iloc[0:indeces[idx], 8] = year_month[idx]
        else:
            df2_unique.iloc[indeces[idx-1]:indeces[idx], 8] = year_month[idx]
            
            
    # from df1 keep only cols of interest
    df1 = df1[
        ['year_month',
         'u',
         'v',
         'wdir',
         'wspd',
         'wind_lat_lon']
    ]
    
    # merge df2_unique with df1
    df2_unique = df2_unique.merge(
        df1,
        left_on=['year_month', 'closest_wind_lat_lon'],
        right_on=['year_month', 'wind_lat_lon'],
        how='left'
    )
    # keep only cols of interest
    df2_unique = df2_unique[
        ['lat',
         'lon',
         'ZCTA10',
         'u',
         'v',
         'wdir',
         'wspd',
         'year_month']
    ]
    
    df2_unique.dropna(
        inplace=True
    )
    
    df2_unique.reset_index(
        drop=True,
        inplace=True
    )
    
    df2_unique.drop_duplicates(
    ['year_month', 'ZCTA10'],
    inplace=True
    )

    df2_unique.reset_index(
        drop=True,
        inplace=True
    )
    
    return df2_unique

### Step 4: Read data

In [80]:
zip_poly = read_census_geom()

In [81]:
for year in range(1991,2023):
    if os.path.exists(out_dir + str(year) + 'winds.csv'):
        print('- File ' + str(year) + 'winds.csv already exists. -')
    else:
        print(f'----- Starting on year {year} -----')
        df = read_clean_wind(year)
        df_final = add_zcta_to_wind(df, zip_poly)
        df_final.to_csv(out_dir  + str(year) + 'winds.csv')
        print('--- Now saving file' + out_dir  + str(year) + 'winds.csv ---')

- File 1991winds.csv already exists. -
- File 1992winds.csv already exists. -
- File 1993winds.csv already exists. -
- File 1994winds.csv already exists. -
- File 1995winds.csv already exists. -
- File 1996winds.csv already exists. -
- File 1997winds.csv already exists. -
- File 1998winds.csv already exists. -
- File 1999winds.csv already exists. -
- File 2000winds.csv already exists. -
- File 2001winds.csv already exists. -
- File 2002winds.csv already exists. -
- File 2003winds.csv already exists. -
- File 2004winds.csv already exists. -
- File 2005winds.csv already exists. -
- File 2006winds.csv already exists. -
- File 2007winds.csv already exists. -
- File 2008winds.csv already exists. -
- File 2009winds.csv already exists. -
- File 2010winds.csv already exists. -
- File 2011winds.csv already exists. -
- File 2012winds.csv already exists. -
- File 2013winds.csv already exists. -
- File 2014winds.csv already exists. -
- File 2015winds.csv already exists. -
- File 2016winds.csv alre

``census geom``

In [92]:
import glob
files = glob.glob(out_dir+"*.csv")

df_all_years = pd.DataFrame()
for f in files:
    csv = pd.read_csv(f)
    df_all_years = df_all_years.append(csv)

In [96]:
df_all_years['year_month']=df_all_years['year_month'].astype(str)
df_all_years['year']=df_all_years['year_month'].str.slice(start=0,stop=4)
df_all_years['month']=df_all_years['year_month'].str.slice(start=4,stop=6)

df_all_years.to_csv(out_dir+"all_years_wind_data.csv")
df_all_years

,Unnamed: 0,lat,lon,ZCTA10,u,v,wdir,wspd,year_month,year,month
0,0,37.465,-117.936,89010,0.992558,0.124684,7.159901,1.000358,200601,2006,01
1,1,35.396,-116.322,89019,0.088867,-0.147450,301.077087,0.172160,200601,2006,01
2,2,36.161,-116.139,89060,-0.319009,-0.046086,188.220367,0.322321,200601,2006,01
3,3,35.957,-115.897,89061,-0.106072,-0.459862,257.011322,0.471937,200601,2006,01
4,4,39.520,-120.032,89439,0.707161,0.798291,48.464073,1.066463,200601,2006,01
...,...,...,...,...,...,...,...,...,...,...,...
19591,19591,39.061,-120.210,96145,0.370858,0.799831,65.124237,0.881626,199412,1994,12
19592,19592,39.149,-120.248,96146,0.370858,0.799831,65.124237,0.881626,199412,1994,12
19593,19593,39.236,-120.062,96148,0.370858,0.799831,65.124237,0.881626,199412,1994,12
19594,19594,38.732,-120.033,96150,-0.001801,0.435539,90.236954,0.435543,199412,1994,12
